In [13]:
import pandas as pd
import requests
import json
import numpy
import time
from datetime import date
#import requests_cache
from IPython.core.display import clear_output
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# get auth Token

body = {"email": "p____o@gmail.com"}

headers = {
    'User-Agent': 'Bird/4.119.0(co.bird.Ride; build:3; iOS 14.3.0) Alamofire/5.2.2',
    'Device-Id': 'BC5BDA72-12A2-5DF4-BD11-92ACF8FFC86D', # is a random 16-bit GUID
    'Platform': 'ios',
    'App-Version': '4.119.0',
    'Content-Type': 'application/json'
    }

POST = 'https://api-auth.prod.birdapp.com/api/v1/auth/email'

r = requests.post(url=POST, headers=headers,json=body)

r.status_code

200

In [3]:
r.json()

{'validation_required': True}

In [4]:
# get 'access' and 'refresh' codes

headers = {
    'User-Agent': 'Bird/4.119.0(co.bird.Ride; build:3; iOS 14.3.0) Alamofire/5.2.2',
    'Device-Id': 'BC5BDA72-12A2-5DF4-BD11-92ACF8FFC86D', # is a random 16-bit GUID
    'Platform': 'ios',
    'App-Version': '4.119.0',
    'Content-Type': 'application/json'
    }

tkn = {"token": "DidHSRECQxeCJFhgyW9CS2A"}

POST = "https://api-auth.prod.birdapp.com/api/v1/auth/magic-link/use"

r = requests.post(url=POST, headers=headers,json=tkn)

r.status_code

200

In [5]:
j = r.json()

## Refresh Auth Token

By default bird tokens expire after one day. They can be easily refreshed without having to get another magic link. Send a POST request to ***https://api-auth.prod.birdapp.com/api/v1/auth/refresh/token*** with the following headers:
* User-Agent: Bird/4.119.0(co.bird.Ride; build:3; iOS 14.3.0) Alamofire/5.2.2
* Device-Id: 'BC5BDA72-12A2-5DF4-BD11-92ACF8FFC86D'
* Platform: ios
* App-Version: 4.119.0
* Content-Type: application/json
* Authorization: Bearer + j["refresh"]

In [21]:
# refresh auth token

POST = " https://api-auth.prod.birdapp.com/api/v1/auth/refresh/token"

headers = {
    'User-Agent': 'Bird/4.119.0(co.bird.Ride; build:3; iOS 14.3.0) Alamofire/5.2.2',
    'Device-Id': '4984dbee-5300-40d6-bbdf-39c94ae4a39e',
    'Platform': 'ios',
    'App-Version': '4.119.0',
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + j["refresh"]
    }

r = requests.post(url=POST, headers=headers)
r.status_code

200

In [15]:
# location request

def make_a_request(lat,lon,rad):
    
    # set headers
    headers = {
        'Authorization': 'Bearer '+j['access'],
        'User-Agent': 'Bird/4.119.0(co.bird.Ride; build:3; iOS 14.3.0) Alamofire/5.2.2',
        'legacyrequest': 'false',
        'Device-Id': 'BC5BDA72-12A2-5DF4-BD11-92ACF8FFC86D',
        'App-Version': '4.119.0',        
        'Location': json.dumps({"latitude": lat, "longitude": lon,"altitude":500,"accuracy":100,"speed":-1,"heading":-1}, sort_keys=True)
    }
    # set url
    URL = 'https://api-bird.prod.birdapp.com/bird/nearby?latitude=' + str(lat) + '&longitude=' + str(lon) + '&radius=' + str(rad)
    
    # make a GET request!
    r = requests.get(url=URL, headers=headers)
    
    return r

In [34]:
# load the list of coordinates for the requests

coords = pd.read_csv('./Downloaded_data/bird_data/request_location.csv')
coords = coords.drop(['Unnamed: 0'],axis=1)

In [40]:
# create a list of dict: each dict contains the coordinates for a request

coords_list = []

for i in range(len(coords)):
    
    coord_dict = {'latitude': coords.iloc[i]['latitude'], 'longitude': coords.iloc[i]['longitude']}
    
    coords_list.append(coord_dict)

In [ ]:
#collecting data

responses = []

start_time = time.time()

# set the radius for the requests
R = 500

# set the duration (in seconds) of data collecting
duration = 691200

end_time = start_time + duration

start_day = '2021-05-24'
end_day = '2021-06-03'

request_counter = 0

print('Data collection started at: ', time.ctime(start_time))

print('Data collection will end at: ', time.ctime(end_time))

# make the requests

while time.time() < end_time:
#while request_counter < 1:
    
    #print('Data collection started at: ', time.ctime(start_time))

    #print('Data collection will end at: ', time.ctime(end_time))
    
    #print('***'*15)
    
    #print("Group of requests number ",request_counter)
    
    dfs = []
    
    for i in range(len(coords_list)):
        
        
        
        # make the API call
        response = make_a_request(coords_list[i]['latitude'],coords_list[i]['longitude'],R)
        
        # if we get an error, print the response and halt the loop
        if response.status_code != 200:
            print(response.text)
            break
        
        # create a df with the data of the request
        r_json = response.json()
        r_json_bike = r_json['birds']
        r_df = pd.DataFrame(r_json_bike)
        
        # add to the df WHEN was made the request
        t = time.time()        
        r_df['Time'] = t
        
        # add to the df WHERE was made the request
        r_df['Request Latitude'] = coords_list[i]['latitude']
        r_df['Request Longitude'] = coords_list[i]['longitude']
        
        # rearrange the latitude and longitude data
        lats = []
        lons = []

        for k in range(len(r_df)):
            temp_df = pd.DataFrame(r_json['birds'][k])
            lats.append(temp_df.iloc[0]['location'])
            lons.append(temp_df.iloc[1]['location'])
    
        r_df['latitude'] = lats
        r_df['longitude'] = lons
        
        # delete the 'location' column
        r_df=r_df.drop(['location'], axis=1)
        
        #sort the columns of the df
        r_df=r_df[['id','latitude','longitude','code','model','vehicle_class','captive','partner_id','battery_level','estimated_range','area_key','nest_id','bounty_id','Time','Request Latitude','Request Longitude']]
        
        # append df to a list
        dfs.append(r_df)
        
        # print some output so we can see the status
        #print("Location of the last request: {}".format(coord_labels[i]))
        #print('Time of the last request: {}'.format(time.ctime(time.time())))
        text_file = open("Updates.txt", "w")
        n = text_file.write('# '+str(i)+' - Time of the last request: ' + str(time.ctime(time.time())))
        text_file.close()
        
        
        # append response to a list
        responses.append(response)
        
        #if it's not a cached result, sleep
        #if not getattr(response,'from_cache',False):
        time.sleep(25)
        
    # create a single df for each group of requests
    df = pd.concat(dfs)
    
    # export the df as a .csv file
    df.to_csv('./Downloaded_data/bird_data/TimeSlot'+str(request_counter)+'_Start'+str(start_day)+'_End'+str(end_day)+'.csv')
    
    request_counter = request_counter + 1
    
    #between each group of requests, sleep 50 min 
    time.sleep(2935)
    
    
    
    # clear the output to make things neater
    #clear_output(wait = True)

Data collection started at:  Wed May 26 14:57:10 2021
Data collection will end at:  Thu Jun  3 14:57:10 2021


## 2021-05-24 small data collection

In [ ]:
# try the API using this 5 coordinates


# requests_cache.install_cache()

# coordinates of comune di torino (anagrafe)
coords_CT = {"latitude": 45.0781, "longitude": 7.6761 }

# coordinates of Torino PN
coords_PN = {"latitude": 45.05, "longitude": 7.683333 }

#coordinates of Torino PS
coords_PS = {"latitude": 45.066667, "longitude": 7.666667 }

#coordinates of Piazza Vittorio Veneto
coords_PVV = {"latitude": 45.0650470102, "longitude": 7.69518884942 }

#coordinates of Torino Lingotto (Train Station)
coords_TL = {"latitude": 45.066667, "longitude": 7.65 }

coords_list = [coords_CT,coords_PN,coords_PS,coords_PVV,coords_TL]
coord_labels = ['Comune','Porta Nuova','Porta Susa','Piazza Vittorio','Lingotto']

In [147]:
# create a single dataframe with all the collected data during the day 2021-05-24

frames = []

for i in range(12):
    frames.append(pd.read_csv('./Downloaded_data/bird_data/Group'+str(i)+'_DATE2021-05-24.csv'))
    
global_frame = pd.concat(frames)


print('Number of different e-scooters detected during the day in all areas: ', global_frame['id'].unique().shape[0])

Number of different e-scooters detected during the day in all areas:  1086


In [143]:
"""
This is a test to verify that the IDs of the e-scooters, don't change if someone uses the e-scooter.
The following rows of code identifies all the e-scooters's ID that change position in the df's index,
Then, I picked some of them and I verified that the ID of the e-scooter doesn

"""

un_id = global_frame['id'].unique()

for idx in un_id:
    
    found = []
    pr = False
    for k in range(len(frames)):
        
        found.append(getIndexes(frames[k], idx))
        
        if k>0:
            if len(found[k]) > 0:
                if len(found[k-1]) > 0:
                    if np.abs(found[k-1][0][0]-found[k][0][0])>40:
                        pr = True
                        print('******'*15)
    if pr:
        print(found)
    

******************************************************************************************
******************************************************************************************
[[(243, 'id'), (502, 'id'), (1228, 'id')], [(243, 'id'), (506, 'id'), (1226, 'id')], [(246, 'id'), (501, 'id'), (1218, 'id')], [(504, 'id'), (1215, 'id')], [(242, 'id'), (504, 'id'), (1215, 'id')], [(234, 'id'), (508, 'id'), (1216, 'id')], [(237, 'id'), (507, 'id'), (1212, 'id')], [(231, 'id'), (507, 'id'), (1212, 'id')], [(227, 'id'), (501, 'id'), (1208, 'id')], [(231, 'id'), (501, 'id'), (1202, 'id')], [(229, 'id'), (502, 'id'), (1206, 'id')], [(243, 'id'), (503, 'id'), (1207, 'id')]]
******************************************************************************************
******************************************************************************************
[[(248, 'id'), (501, 'id'), (1230, 'id')], [(240, 'id'), (508, 'id'), (1224, 'id')], [(245, 'id'), (503, 'id'), (1219, 'id')], [(502, 'id'), (12

******************************************************************************************
******************************************************************************************
[[(655, 'id'), (970, 'id')], [(651, 'id'), (973, 'id')], [(648, 'id'), (969, 'id')], [(646, 'id'), (971, 'id')], [(648, 'id'), (963, 'id')], [(651, 'id'), (962, 'id')], [(647, 'id'), (958, 'id')], [(248, 'id'), (647, 'id'), (955, 'id')], [(243, 'id'), (642, 'id'), (952, 'id')], [(239, 'id'), (640, 'id'), (948, 'id')], [(241, 'id'), (639, 'id'), (949, 'id')], [(645, 'id'), (961, 'id')]]
******************************************************************************************
******************************************************************************************
[[(696, 'id'), (947, 'id')], [(690, 'id'), (947, 'id')], [(679, 'id'), (945, 'id')], [(684, 'id'), (947, 'id')], [(684, 'id'), (942, 'id')], [(499, 'id'), (686, 'id'), (941, 'id')], [(493, 'id'), (681, 'id'), (937, 'id')], [(493, 'id'), (680, 'id'

******************************************************************************************
******************************************************************************************
[[], [], [], [], [], [(748, 'id'), (911, 'id')], [(734, 'id'), (917, 'id')], [(729, 'id'), (918, 'id')], [(723, 'id'), (915, 'id')], [(247, 'id'), (723, 'id'), (911, 'id')], [(722, 'id'), (911, 'id')], [(727, 'id'), (925, 'id')]]
******************************************************************************************
[[], [], [], [], [], [], [(523, 'id')], [(523, 'id')], [(247, 'id'), (523, 'id')], [(243, 'id'), (523, 'id')], [(244, 'id'), (522, 'id')], []]
******************************************************************************************
[[], [], [], [], [], [], [], [(246, 'id'), (510, 'id'), (1194, 'id')], [(244, 'id'), (510, 'id'), (1189, 'id')], [(242, 'id'), (510, 'id'), (1186, 'id')], [(238, 'id'), (510, 'id'), (1192, 'id')], [(510, 'id'), (1193, 'id')]]
*************************************

In [144]:
frames[0].iloc[705]

Unnamed: 0                                           205
id                  d30db272-918c-46af-bd77-4497847e5490
latitude                                       45.070633
longitude                                       7.656248
Request Location                              Porta Susa
code                                               F9H••
model                                                 rf
vehicle_class                                    scooter
captive                                            False
partner_id          5d6c37aa-9966-4532-958a-326cd62932df
battery_level                                         89
estimated_range                                    20358
area_key                B7B84C04C63BD5ED07794DFF7450725D
nest_id             784e6a6e-c617-4c69-a854-01d969f3caa6
bounty_id                                            NaN
Time                                   1621853636.774098
Name: 705, dtype: object

In [145]:
frames[1].iloc[237]

Unnamed: 0                                           237
id                  d30db272-918c-46af-bd77-4497847e5490
latitude                                        45.06554
longitude                                       7.677283
Request Location                                  Comune
code                                               F9H••
model                                                 rf
vehicle_class                                    scooter
captive                                            False
partner_id          5d6c37aa-9966-4532-958a-326cd62932df
battery_level                                         74
estimated_range                                    16965
area_key                B7B84C04C63BD5ED07794DFF7450725D
nest_id             928377cb-1d1a-46f5-b1fe-74fd923b5d0d
bounty_id                                            NaN
Time                                   1621861204.503018
Name: 237, dtype: object

In [87]:
# This function will return a list of
# positions where element exists
# in the dataframe.
def getIndexes(dfObj, value):
      
    # Empty list
    listOfPos = []
      
    # isin() method will return a dataframe with 
    # boolean values, True at the positions    
    # where element exists
    result = dfObj.isin([value])
      
    # any() method will return 
    # a boolean series
    seriesObj = result.any()
  
    # Get list of column names where 
    # element exists
    columnNames = list(seriesObj[seriesObj == True].index)
     
    # Iterate over the list of columns and
    # extract the row index where element exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
  
        for row in rows:
            listOfPos.append((row, col))
              
    # This list contains a list tuples with 
    # the index of element in the dataframe
    return listOfPos